In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

from sklearn.compose import ColumnTransformer

import category_encoders as ce
import missingno as msgn

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

from scipy import stats

seed = 123
np.random.seed(seed)

In [2]:
path = r'../input/bank-customer-segmentation/bank_transactions.csv'
df = pd.read_csv(path)

In [3]:
df.sample(5)

In [31]:
df.info()

In [7]:
df.CustGender.unique()

In [5]:
df.describe()

In [6]:
df.describe(include='object')

#### some preprocessing

In [33]:
# fill nans in DOB and replace wrong lengths
df.CustomerDOB.fillna(df.CustomerDOB.mode()[0], inplace=True)
df.loc[df.CustomerDOB == '1/1/1800', 'CustomerDOB'] = '1/1/18'

In [41]:
# add century to year
df['CustomerDOB'] = df['CustomerDOB'].str[:-2] + '19' + df['CustomerDOB'].str[-2:]

In [44]:
# convert DOB to datetime
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], dayfirst=True)

In [53]:
# calculate customer age
df['CustomerAge'] = df["CustomerDOB"].apply(lambda x : (pd.datetime.now().year - x.year))

In [56]:
# replace age > 90 by median
df.loc[df['CustomerAge'] > 90, 'CustomerAge'] = df['CustomerAge'].median()

In [58]:
df['TransactionTime'] = df['TransactionTime'].astype(np.int64)

In [61]:
df['CustLocation'].fillna(df['CustLocation'].mode()[0], inplace=True)

In [62]:
df['CustAccountBalance'].fillna(0, inplace=True)

In [63]:
df['CustGender'].fillna(df['CustGender'].mode()[0], inplace=True)

In [66]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], dayfirst=True)

In [70]:
df['TransactionDay'] = df['TransactionDate'].dt.day.astype(np.int32)
df['TransactionDayOfWeek'] = df['TransactionDate'].dt.dayofweek.astype(np.int32)

In [73]:
df.drop(['CustomerDOB', 'TransactionDate'], axis=1, inplace=True)

In [75]:
# inspect for outliers to aggregate correctly
numerical_cols = df.select_dtypes(exclude=['object']).columns.tolist()

for idx, column in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, idx)
    sns.boxplot(y=df[column], data=df)
    plt.title(f'{column}')
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.tight_layout()
plt.show()

In [80]:
# to large rare values (outlier) may significantly affect KMeans as it calculates distancies
df.drop(df[df['TransactionAmount (INR)'] > 1e6].index, axis=0, inplace=True) 

In [84]:
df_grouped = df.groupby('CustomerID').agg({'TransactionID': 'count', 'CustGender': lambda x: x.mode()[0], 'CustLocation': lambda x: x.mode()[0], 'CustAccountBalance': 'mean', 'TransactionTime': 'mean', 'TransactionAmount (INR)': 'mean', 'CustomerAge': 'max', 'TransactionDay': 'mean', 'TransactionDayOfWeek': 'mean'})
# cust id to index

In [3]:
def reduce_memory(df, verbose=0):
    if verbose != 0:
        start_mem = df.memory_usage().sum() / 1024 ** 2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object and str(col_type)[:4] != 'uint' and str(col_type) != 'category':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif str(col_type)[:4] != 'uint':
            df[col] = df[col].astype('category')
    if verbose != 0:
        end_mem = df.memory_usage().sum() / 1024 ** 2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [87]:
df_grouped = reduce_memory(df_grouped)

In [94]:
df_grouped.to_csv('table.csv', index=True, columns=df_grouped.columns, encoding='utf-8')

In [4]:
new_df = pd.read_csv('../input/clustering-table/table.csv')

In [5]:
new_df.set_index('CustomerID', drop=True, inplace=True)
new_df.sample(5)

In [6]:
new_df = reduce_memory(new_df)

In [104]:
plt.figure(figsize = (14,12))
corr_matrix = new_df.corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0
sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')
plt.title('Correlation matrix')
plt.show()

In [7]:
numerical_cols = new_df.select_dtypes(exclude=['category']).columns.tolist()
onehot_cols = ['CustGender',]
enc_cols = ['CustLocation',]

In [8]:
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import LabelEncoder
locations = LabelEncoder().fit_transform(new_df['CustLocation'])
labels = np.unique(locations)

weights = compute_class_weight(class_weight='balanced', classes=labels, y=locations)
weights

In [9]:
loc_dict = (new_df['CustLocation'].value_counts(normalize=True) * weights).to_dict()
new_df['CustLocation'] = new_df['CustLocation'].map(loc_dict)

In [10]:
new_df = pd.get_dummies(new_df, prefix=['Gender'])

In [12]:
df_columns = new_df.columns.tolist()
df_index = new_df.index

In [13]:
column_transformer = ColumnTransformer(transformers=[('scaler', MinMaxScaler(), numerical_cols)],
                                       remainder='passthrough')

In [14]:
X = column_transformer.fit_transform(new_df)
X = pd.DataFrame(data=X, columns=df_columns, index=df_index)
X.head()

In [15]:
X = reduce_memory(X)
# now the data is ready to clustering

In [143]:
distortions = [] 
max_k = 10
for i in range(1, max_k+1):
    if len(X) >= i:
        model = KMeans(n_clusters=i, max_iter=300, n_init=10)
        model.fit(X)
        distortions.append(model.inertia_)
    

k = [i*100 for i in np.diff(distortions, 2)].index(min([i*100 for i in np.diff(distortions,2)]))


fig, ax = plt.subplots()
ax.plot(range(1, len(distortions)+1), distortions)
ax.axvline(k, ls='-', color="red", label="k = " + str(k))
ax.set(title='The Elbow Method', xlabel='Number of clusters', ylabel="Distortion")

plt.show()

In [16]:
new_df['cluster'] = KMeans(n_clusters=4, max_iter=300, n_init=10).fit_predict(X)
new_df.sample(10)

In [17]:
# CHANGE for me and inspect
columns = ["Cluster", "Sum_transactions", "Mean_Balance", "Mean_Transaction", "Mean_age"]
result_df = pd.DataFrame(columns = columns)
for i in set(new_df['cluster']):
    cluster_df = new_df.loc[new_df['cluster'] == i]
    result_df.loc[i] = [i, cluster_df["TransactionID"].sum(), cluster_df["CustAccountBalance"].mean(), cluster_df["TransactionAmount (INR)"].mean(), cluster_df["CustomerAge"].median()]

In [18]:
result_df

In [19]:
def statistic_output(df, alpha=0.05, sample_size=0):
    data = df.copy()
    data.drop_duplicates(inplace=True)
    if sample_size == 0:
        sample_size = int(0.05 * len(data))
    columns = data.iloc[:, :-4].select_dtypes(exclude=['category', 'object']).columns
    for column in columns:
        samples = []
        for i in range(data['cluster'].nunique()):
            samples.append(data.loc[data['cluster'] == i, column].sample(sample_size, random_state=seed))
        is_normal = np.all([stats.shapiro(sample)[1] >= alpha for sample in samples])
        if is_normal:
            _, pvalue = stats.f_oneway(*samples)
        else:
            _, pvalue = stats.kruskal(*samples)
        if pvalue < alpha:
            print(f'\033[1m{column}\033[0m has diff. distribution in clusters.')
        else:
            print(f'{column} has no statistical difference in clusters')
    del samples

In [20]:
statistic_output(new_df, sample_size=10000)

In [20]:
# DBDCAN
dbscan = DBSCAN(eps=0.85, min_samples=5, metric='euclidean', algorithm='auto', leaf_size=30, p=2, n_jobs=-1)

new_df['cluster'] = dbscan.fit_predict(X)
# RAM runs out of space

In [ ]:
aglster = AgglomerativeClustering(n_clusters=4, affinity='euclidean', memory=None, connectivity=None, 
                                  compute_full_tree='auto', linkage='ward', distance_threshold=None, compute_distances=False)

new_df['cluster'] = aglster.fit_predict(X)